## Molecule Generation using Cetane Number (CN)

In [3]:
import os
import sys
import torch
import joblib
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors

from torchdrug import core, datasets, models, tasks
from torch import optim

os.environ["CUDA_VISIBLE_DEVICES"] = ""   # hide GPUs from Torch/TorchDrug

# ============================================================
# 1. Project paths & imports from src/
# ============================================================
import joblib
import sys, os

PROJECT_ROOT = "/home/salvina2004/biofuel-ml"
SRC_DIR = os.path.join(PROJECT_ROOT, "src")

sys.path.append(PROJECT_ROOT)
sys.path.append(SRC_DIR)

print("PYTHONPATH updated:", sys.path)
if PROJECT_ROOT not in sys.path:
    sys.path.append(PROJECT_ROOT)
# ============================================================
# DEVICE CONFIGURATION
# ============================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ============================================================
# PROJECT SETUP
# ============================================================
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(PROJECT_ROOT)

from src.feature_selection import FeatureSelector, prepare_prediction_features
from src.feature_engineering import *

# Load trained model + selector
model = joblib.load(os.path.join(PROJECT_ROOT, "models/extratrees_model.pkl"))
selector = joblib.load(os.path.join(PROJECT_ROOT, "models/feature_selector.pkl"))

# ============================================================
# CN PREDICTION
# ============================================================
def predict_cn(smiles: str) -> float:
    X = prepare_prediction_features([smiles], selector)
    return float(model.predict(X)[0])


# ============================================================
# REWARD FUNCTIONS (GPU-SAFE)
# ============================================================
def reward_maximize_cn(smiles_list, device=device):
    rewards = []
    for smi in smiles_list:
        cn = predict_cn(smi)
        rewards.append(min(cn / 120.0, 1.0))
    return torch.tensor(rewards, dtype=torch.float32, device=device)


def reward_target_range_cn(smiles_list, target_min=50, target_max=70, device=device):
    rewards = []
    for smi in smiles_list:
        cn = predict_cn(smi)
        if target_min <= cn <= target_max:
            rewards.append(1.0)
        else:
            d = target_min - cn if cn < target_min else cn - target_max
            rewards.append(float(np.exp(-d / 20.0)))
    return torch.tensor(rewards, dtype=torch.float32, device=device)


def reward_multi_objective(smiles_list, target_cn=60, device=device):
    rewards = []
    for smi in smiles_list:
        mol = Chem.MolFromSmiles(smi)
        if mol is None:
            rewards.append(0.0)
            continue

        cn = predict_cn(smi)
        cn_r = max(0, 1 - abs(cn - target_cn) / 100)

        mw = Descriptors.MolWt(mol)
        mw_r = 1.0 if 150 < mw < 300 else max(0, 1 - abs(mw - 225) / 200)

        logp = Descriptors.MolLogP(mol)
        logp_r = 1.0 if 2 < logp < 6 else max(0, 1 - abs(logp - 4) / 5)

        rewards.append(0.7 * cn_r + 0.2 * mw_r + 0.1 * logp_r)

    return torch.tensor(rewards, dtype=torch.float32, device=device)


# ============================================================
# CUSTOM GCPN TASK — REWARD ON GPU
# ============================================================
class CNOptimizationTask(tasks.GCPNGeneration):
    def __init__(self, model, atom_types,
                 optimization_goal="maximize",
                 target_min=50, target_max=70,
                 target_cn=60,
                 **kwargs):

        super().__init__(model, atom_types, task="plogp", **kwargs)

        self.optimization_goal = optimization_goal
        self.target_min = target_min
        self.target_max = target_max
        self.target_cn = target_cn

    def get_reward(self, graph, smiles_list):
        if self.optimization_goal == "maximize":
            return reward_maximize_cn(smiles_list, device=device)

        elif self.optimization_goal == "target_range":
            return reward_target_range_cn(smiles_list,
                                          target_min=self.target_min,
                                          target_max=self.target_max,
                                          device=device)

        elif self.optimization_goal == "multi_objective":
            return reward_multi_objective(smiles_list,
                                          target_cn=self.target_cn,
                                          device=device)

        raise ValueError(f"Unknown reward goal: {self.optimization_goal}")


# ============================================================
# TRAINING PIPELINE (NOW GPU-SAFE)
# ============================================================
def run_cn_optimization(optimization_goal="maximize",
                        target_cn=60,
                        target_min=50, target_max=70,
                        num_epochs=1,
                        batch_size=8,
                        lr=1e-5):

    print("\n==============================")
    print("     CN OPTIMIZATION START    ")
    print("==============================")

    dataset = datasets.ZINC250k("./zinc_data", kekulize=True, atom_feature="symbol")

    gcpn = models.RGCN(
        input_dim=dataset.node_feature_dim,
        num_relation=dataset.num_bond_type,
        hidden_dims=[256, 256, 256, 256],
        batch_norm=False
    ).to(device)

    task = CNOptimizationTask(
        gcpn,
        dataset.atom_types,
        optimization_goal=optimization_goal,
        target_cn=target_cn,
        target_min=target_min,
        target_max=target_max,
        max_edge_unroll=12,
        max_node=38,
        criterion="ppo",
        reward_temperature=1.0,
        agent_update_interval=3,
        gamma=0.9,
    ).to(device)

    optimizer = optim.Adam(task.parameters(), lr=lr)

    solver = core.Engine(
        task,
        dataset,
        None,
        None,
        optimizer,
        gpus=(0,) if torch.cuda.is_available() else None,
        batch_size=batch_size,
        log_interval=10
    )

    print("\n Starting RL fine-tuning...")
    solver.train(num_epoch=num_epochs)

    solver.save("gcpn_cn_optimized.pkl")
    print("\n Saved: gcpn_cn_optimized.pkl")

    return solver, task


# ============================================================
# MOLECULE GENERATION
# ============================================================
def generate_molecules(task, num_samples=20):
    task.eval()

    with torch.no_grad():
        graphs = task.generate(num_sample=num_samples)

    results = []
    for g in graphs:
        smi = g.to_smiles()
        mol = Chem.MolFromSmiles(smi)
        cn = predict_cn(smi)

        results.append({
            "smiles": smi,
            "cn": cn,
            "mw": Descriptors.MolWt(mol) if mol else None,
            "logp": Descriptors.MolLogP(mol) if mol else None
        })

    return results


def display_results(results, top_k=10):
    df = pd.DataFrame(results)
    df_sorted = df.sort_values("cn", ascending=False)
    print(df_sorted.head(top_k))
    df_sorted.to_csv("generated_molecules_cn.csv", index=False)
    return df_sorted


# ============================================================
# MAIN
# ============================================================
if __name__ == "__main__":

    solver, task = run_cn_optimization(
        optimization_goal="maximize",
        num_epochs=1,
        batch_size=8
    )

    results = generate_molecules(task, num_samples=10)
    display_results(results)


PYTHONPATH updated: ['/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/salvina2004/biofuel-ml/torchdrug_env/lib/python3.10/site-packages', '/home/salvina2004/biofuel-ml', '/home/salvina2004/biofuel-ml/src', '/home/salvina2004/biofuel-ml', '/home/salvina2004/biofuel-ml', '/home/salvina2004/biofuel-ml/src', '/home/salvina2004/biofuel-ml', '/home/salvina2004/biofuel-ml', '/home/salvina2004/biofuel-ml/src']
Using device: cuda

     CN OPTIMIZATION START    


Loading ./zinc_data/250k_rndm_zinc_drugs_clean_3.csv:  50%|█████     | 249456/498911 [00:01<00:01, 156342.09it/s]
Constructing molecules from SMILES: 100%|██████████| 249455/249455 [03:52<00:00, 1074.07it/s]


23:56:39   Preprocess training set
23:56:39   {'batch_size': 8,
 'class': 'core.Engine',
 'gpus': (0,),
 'gradient_interval': 1,
 'log_interval': 10,
 'logger': 'logging',
 'num_worker': 0,
 'optimizer': {'amsgrad': False,
               'betas': (0.9, 0.999),
               'capturable': False,
               'class': 'optim.Adam',
               'differentiable': False,
               'eps': 1e-08,
               'foreach': None,
               'fused': None,
               'lr': 1e-05,
               'maximize': False,
               'weight_decay': 0},
 'scheduler': None,
 'task': {'agent_update_interval': 3,
          'atom_types': [6, 7, 8, 9, 15, 16, 17, 35, 53],
          'baseline_momentum': 0.9,
          'class': 'tasks.GCPNGeneration',
          'criterion': 'ppo',
          'gamma': 0.9,
          'hidden_dim_mlp': 128,
          'max_edge_unroll': 12,
          'max_node': 38,
          'model': {'activation': 'relu',
                    'batch_norm': False,
             

KeyboardInterrupt: 